In [4]:
import pandas as pd
import numpy as np

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

from IPython.display import display
show = display

from sklearn.impute import SimpleImputer 
from sklearn.preprocessing import OneHotEncoder

from sklearn.model_selection import KFold
import sklearn.metrics as skm

lda, liwc, mtf = pd.DataFrame(), pd.DataFrame(), pd.DataFrame()
train = pd.DataFrame()

# TODO add city as feature
for city in ["NewYork", "London", "Singapore"]:
    lda = pd.concat([lda, pd.read_csv(f"data/Features/features{city}/Twitter/LDA50Features.csv", encoding="ISO-8859-15")], axis=0, ignore_index=True)
    liwc = pd.concat([liwc, pd.read_csv(f"data/Features/features{city}/Twitter/LIWCFeatures.csv", encoding="ISO-8859-15")], axis=0, ignore_index=True)
    mtf = pd.concat([mtf, pd.read_csv(f"data/Features/features{city}/Twitter/manuallyDefinedTextFeatures.csv", encoding="ISO-8859-15")], axis=0, ignore_index=True)

    train = pd.concat([train, pd.read_csv("data/NYTrain.csv")], axis=0, ignore_index=True)

lda = lda.drop_duplicates(subset=["_id"], ignore_index=True)
liwc = liwc.drop_duplicates(subset=["_id"], ignore_index=True)
mtf = mtf.drop_duplicates(subset=["_id"], ignore_index=True)

# Gender prediction

from sklearn.ensemble import RandomForestClassifier
import joblib

with joblib.parallel_backend("multiprocessing"): 

    for data, feature_name in zip([lda, liwc, mtf], ["LDA50Features", "LIWCFeatures", "ManuallyDefinedTextFeatures"]):
        show(feature_name)
        
        df = pd.concat([data, train["gender"]], axis=1, join='inner')

        df = df.dropna(subset=["gender"])

        Y = df["gender"].values

        X = df.drop(["gender", "_id"], axis=1).values


#         for N in [3, 5, 10, 20, 30, 50, 100, 300, 500]:
#             for D in [3, 5, 10, 20, 30, 50, 100, 300, 500]:
#                 show(f"N={N}, D={D}")
        fscore, accuracy = [], []
        fold10 = KFold(n_splits=10, shuffle=True, random_state=42)
        for train_ix, test_ix in fold10.split(X, Y):
            clf = RandomForestClassifier(n_estimators=30, max_depth=300, min_samples_split=2, random_state=42)
            clf.fit(X[train_ix], Y[train_ix])
            y_true, y_pred = Y[test_ix], clf.predict(X[test_ix])
    #         show(y_true)
    #         show(y_pred)
            accuracy.append(skm.accuracy_score(y_true, y_pred))
            fscore.append(skm.f1_score(y_true, y_pred, average="weighted"))

        accuracy = np.array(accuracy).mean(axis=0)
        fscore = np.array(fscore).mean(axis=0)
        show(f"accur: {accuracy}, f1score: {fscore}")




'LDA50Features'

'accur: 0.5518651432444536, f1score: 0.5161125171101468'

'LIWCFeatures'

'accur: 0.5634185013495357, f1score: 0.525913581427609'

'ManuallyDefinedTextFeatures'

'accur: 0.550393967635347, f1score: 0.5186807169161812'